In [67]:
import networkx as nx
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [68]:
# shortest paths and star routing solution selection
from funcs_shortestpaths_stars import __add_node_capacities,__find_disjoint_path_edges,disjoint_path_priority,\
                                      find_disjoint_path_edges,make_star_from_paths,is_valid_star,get_source_and_star,\
                                      users_connected_center_disj, users_connected, check_disjoint_flow

In [69]:
real_topo_users = {0: (4, 22, 25, 41), 1: (1, 3, 5, 6), 2: (1, 7, 15, 17), 3: (3, 7, 9, 11), 4: (11, 13, 17, 18), 5: (1, 7, 9, 11), 6: (2, 3, 10, 11), 7: (2, 16, 17, 19), 8: (1, 2, 11, 12), 9: (7, 16, 20, 22), 10: (2, 8, 15, 16), 11: (5, 116, 133, 138), 12: (5, 13, 15, 17), 13: (5, 13, 15, 23), 14: (5, 13, 18, 19), 15: None, 16: (1, 6, 8, 12), 17: (18, 21, 37, 41), 18: (3, 8, 9, 15), 19: (1, 2, 3, 6), 20: (2, 10, 11, 14), 21: (1, 13, 14, 30), 22: (1, 7, 12, 25), 23: (26, 11, 23, 21), 24: (3, 4, 6, 7), 25: (2, 3, 5, 6), 26: (16, 17, 19, 23), 27: (8, 28, 40, 49), 28: (14, 15, 16, 21), 29: (11, 35, 61, 63), 30: (1, 9, 69, 95), 31: (4, 6, 10, 11), 32: (2, 10, 13, 15), 33: (1, 7, 13, 15), 34: (1, 2, 47, 48), 35: (1, 2, 3, 26), 36: (10, 21, 24, 29), 37: (5, 6, 15, 21), 38: (1, 2, 8, 9), 39: (1, 4, 13, 14), 40: (1, 5, 11, 17), 41: (16, 17, 19, 23), 42: (5, 11, 14, 15), 43: (18, 24, 28, 29), 44: (2, 3, 8, 9), 45: (8, 10, 16, 19), 46: (6, 9, 11, 12), 47: (2, 11, 16, 30), 48: (9, 10, 11, 13), 49: (8, 35, 96, 97), 50: (13, 23, 27, 31), 51: (3, 4, 14, 18), 52: (26, 35, 46, 52), 53: (8, 9, 10, 11), 54: (4, 5, 17, 25), 55: (24, 29, 30, 45), 56: (1, 4, 5, 6), 57: (4, 11, 13, 35), 58: (11, 18, 20, 28), 59: (4, 13, 14, 26), 60: (5, 7, 8, 9), 61: (1, 2, 23, 25), 62: (1, 3, 10, 14), 63: (9, 22, 42, 43), 64: (1, 5, 17, 18), 65: (1, 4, 10, 13), 66: (1, 2, 4, 5), 67: (12, 16, 18, 19), 68: (1, 7, 13, 19), 69: (2, 5, 19, 26), 70: (9, 12, 22, 34), 71: (9, 22, 31, 51), 72: (9, 13, 17, 20), 73: (2, 7, 12, 14), 74: (3, 8, 15, 17), 75: (1, 6, 11, 13), 76: (8, 9, 16, 18), 77: (1, 13, 15, 32), 78: (1, 2, 5, 6), 79: (1, 2, 24, 25), 80: (1, 11, 12, 18), 81: (3, 13, 15, 27), 82: (4, 6, 7, 8), 83: (4, 5, 9, 12), 84: (1, 4, 5, 14), 85: (2, 3, 18, 19), 86: (2, 5, 8, 10), 87: (1, 2, 7, 10), 88: (15, 19, 25, 30), 89: (1, 11, 14, 17), 90: (4, 9, 10, 16), 91: (6, 7, 9, 10), 92: (1, 8, 15, 19), 93: (8, 28, 40, 49), 94: (1, 2, 8, 27), 95: (2, 3, 11, 21), 96: (8, 12, 16, 18), 97: (6, 11, 13, 20), 98: (5, 8, 12, 19), 99: (9, 11, 22, 23), 100: (3, 6, 9, 10), 101: (2, 12, 16, 23), 102: (1, 4, 6, 7), 103: (2, 5, 7, 8), 104: (7, 20, 22, 24)}
real_topo_centers = {0: 35, 1: 2, 2: 26, 3: 12, 4: 8, 5: 1, 6: 9, 7: 12, 8: 7, 10: 11, 11: 81, 12: 12, 13: 12, 14: 18, 16: 3, 17: 32, 19: 7, 21: 12, 22: 9, 23: 25, 26: 14, 27: 24, 28: 1, 29: 22, 30: 31, 33: 12, 34: 28, 35: 11, 36: 26, 37: 13, 38: 7, 39: 8, 40: 2, 41: 14, 42: 17, 43: 20, 45: 4, 47: 1, 48: 12, 49: 32, 50: 33, 51: 14, 52: 12, 54: 21, 55: 14, 57: 31, 58: 23, 60: 11, 61: 7, 62: 4, 63: 30, 64: 4, 65: 7, 68: 9, 70: 26, 71: 30, 72: 18, 74: 18, 76: 1, 77: 12, 79: 17, 81: 10, 82: 9, 83: 6, 84: 11, 86: 6, 88: 18, 89: 18, 90: 12, 92: 15, 93: 24, 94: 15, 95: 13, 96: 22, 97: 10, 98: 16, 100: 3, 101: 11, 102: 2, 103: 6, 104: 1}

In [70]:
# these graphs are problematic
bad_indices = []

bad_indices += [9,18,20,24,25,31,32,44,46,53,56,59,66,69,73,75,78,80,85,87,91,99] # rings and multirings
bad_indices += [15]#,23,60,74,103] # caterpillars/paths
bad_indices += [67] # ill defined center



#bad_indices += [1,19,102] # tiny topos

#bad_indices += [52,90] # two separated clusters

#bad_indices += [49,58,77] # most edges short but one or two EXTREMELY long (usually global networks)


In [71]:
def sort_key(s):
    s_list = s.split('_')
    return int(s_list[1])

def network_name(s):
    s_list = s.split('_')
    return s_list[-1][:-5]

nodes_pd = [] 
edges_pd = []

file_names = []

for file in sorted(os.listdir('real_topologies'), key=sort_key): # sorted so that they are accessed in order
    fp = 'real_topologies\\' + file
    xl = pd.ExcelFile(fp)
    df_nodes = pd.read_excel(xl, 'Nodes_' + network_name(file))
    df_edges = pd.read_excel(xl, 'Edges_' + network_name(file))
    nodes_pd.append(df_nodes)
    edges_pd.append(df_edges)
    file_names.append(network_name(file))



topo_no = 105 # some numbers are missing e.g. 41 45 56 69 73 91
real_topo = {}
pos = [{} for _ in range(topo_no)]

for i in range(len(nodes_pd)):
    # initialise graph
    real_topo[i] = nx.Graph()
    
    node_labels = nodes_pd[i]['Node_ID'].tolist()
    node_labels.sort()
    real_topo[i].add_nodes_from(node_labels)

    edge_source = edges_pd[i]['Source'].tolist()
    edge_destin = edges_pd[i]['Destination'].tolist()

    edge_weight = edges_pd[i]['Computed Length (km)'].tolist() # "weight" is length/distance of optical cable

    # scale longest edge to...
    scale_factor = 100
    edge_weight_scaled = [w/max(edge_weight)*scale_factor for w in edge_weight]
    #print(edge_weight_scaled)
    edge_labels = [(edge_source[j],edge_destin[j],edge_weight_scaled[j]) for j in range(len(edge_source))]
    edge_labels.sort()

    real_topo[i].add_weighted_edges_from(edge_labels)

    # Positions
    pos_x = nodes_pd[i]['Longitude'].tolist()
    pos_y = nodes_pd[i]['Latitude'].tolist()
    for k,node in enumerate(node_labels):
        pos[i][node] = (pos_x[k],pos_y[k])

    # assigning probability
    for u, v in real_topo[i].edges():
        L = real_topo[i][u][v]['weight']
        p_op = 1 # operation probability
        p_ph = 10**(-0.2*L/10) # transmission probability, assuming attenuaion of 0.2 dB/km
        real_topo[i][u][v]['p'] = p_op * p_ph
        
    draw_graphs = False
    save_graphs = False
    if draw_graphs:
        nx.draw(real_topo[i], pos=pos[i], with_labels=True, edgecolors='black', node_color='white', font_size=5, node_size = 70)
        weights = nx.get_edge_attributes(real_topo[i],'weight')
        plt.title(f'{i}_{file_names[i]}')
        if save_graphs:
            plt.savefig('images\\real_topo_img\\blank\\' + f'{i}_{file_names[i]}_blank.png', bbox_inches='tight')
        plt.show()

### Protocol implementation

In [72]:
time_limit = 10

# number of timeslots before qubit decays
T_c = np.inf # qubit coherence lifetime

In [73]:
def steiner_best(G,users):
    steiner_kou = nx.algorithms.approximation.steinertree.steiner_tree(G, users, weight='weight',method='kou')
    steiner_mehl = nx.algorithms.approximation.steinertree.steiner_tree(G, users, weight='weight',method='mehlhorn')
    kou_length = sum(G[e[0]][e[1]]['weight'] for e in steiner_kou.edges(data=True))
    mehl_length = sum(G[e[0]][e[1]]['weight'] for e in steiner_mehl.edges(data=True))
    if kou_length < mehl_length:
        return steiner_kou
    else:
        return steiner_mehl

In [134]:
def protocol(prtcl,G,users,algo):            
    if prtcl not in {'SP','MPG','MPC','MPS','MPT','SPS','SPT'}:
        raise ValueError("prtclrithm must be SP, MPS or MPT.")
    
    if not isinstance(G, nx.Graph):
        raise TypeError("G must be a NetworkX graph.")
    
    if not isinstance(users, (list, set, tuple)):
        raise TypeError("users must be a list or set of nodes.")


    # legacy
    if prtcl == 'MPG':
        prtcl = 'MPS'
    elif prtcl == 'MPC':
        prtcl = 'MPT' 
    elif prtcl == 'SP':
        prtcl = 'SPS' 


    paths = prtcl[0:2]
    route = prtcl[-1]


    
    # initialise G'
    G_prime = nx.Graph()
    G_prime.add_nodes_from(G.nodes())

    
    # evan output
    if route == 'S':
        if algo == 'flow_hops':
            evan_output = get_source_and_star(G,users,'hops')
        elif algo == 'flow_length':
            evan_output = get_source_and_star(G,users,'weight')
        elif algo == 'flow_p':
            evan_output = get_source_and_star(G,users,'p')

    # central node used in SP and MPG
    if route == 'S':   
        if algo == 'old':
            central = old_find_center_node(G,users)[0]
        elif algo in ['flow_hops','flow_length','flow_p']:
            central = evan_output[0]

    # SP: entanglement over shortest path edges
    # MP: entangement over all edges
    if paths == 'SP':
        if route == 'S':
            if algo == 'old':
                entangle_edges = find_disjoint_path_edges(G,users,central)
            elif algo in ['flow_hops','flow_length','flow_p']:
                entangle_edges = evan_output[1]
        elif route == 'T':
            entangle_edges = steiner_best(G,users).edges()
             
    elif paths == 'MP':
        entangle_edges = G.edges(data=True)
    

    edge_memory = {}
    hasGHZ = False 
    iter = 0

    #start_time = time.time()
    while not hasGHZ:
        '''
        if time.time() - start_time > time_limit:
                 print(f'took >{time_limit}s')
                 return None
        '''
        
        iter += 1
        new_edges = []
        
        for e in entangle_edges:
            e_tuple = (e[0],e[1])
            if e in G_prime.edges():
                e_mem = edge_memory[e_tuple]
                if e_mem == 0:
                    G_prime.remove_edge(*e)
                elif e_mem > 0:
                    edge_memory[e_tuple] -= 1

            elif e not in G_prime.edges():
                edge_memory[e_tuple] = 0
                
                e_prob = G.get_edge_data(*e_tuple).get('p', 1.0)
                rng = np.random.random()
                if rng < e_prob:
                    if paths == 'SP':
                        G_prime.add_edge(*e_tuple)
                    elif paths == 'MP':
                        G_prime.add_edge(*e_tuple,weight=e[2]['weight'])
                    edge_memory[e_tuple] = T_c

                    new_edges.append(e)
        

        # Plot each time a new edge is found
        plot_new_e = False
        if plot_new_e and len(new_edges)>0:
            nx.draw(G_prime, with_labels=True, edgecolors='black', font_size=5, node_size=70, pos=pos[i],node_color=real_topo_colors[i])
            plt.title(prtcl + ': iteration ' + str(iter))
            plt.show()
        
        # SP,MPG: checks if each user node connects to central node
        # MPC: checks if all user nodes are connected
        if paths == 'SP':
            hasGHZ = all([e in G_prime.edges() for e in entangle_edges])

        elif prtcl == 'MPS':
            if users_connected(G_prime,users):
                if algo == 'old':
                    UCCD = users_connected_center_disj(G_prime,users,central)
                    hasGHZ = UCCD[0]
                    valid_edges = UCCD[1]
                elif algo in ['flow_hops','flow_length','flow_p']:
                    if algo == 'flow_hops':
                        UCCD = check_disjoint_flow(G_prime,users,evan_output[0],'hops')
                    elif algo == 'flow_length':
                        UCCD = check_disjoint_flow(G_prime,users,evan_output[0],'weight')
                    elif algo == 'flow_p':
                        UCCD = check_disjoint_flow(G_prime,users,evan_output[0],'p')
                    
                hasGHZ = UCCD[0]
                valid_edges = UCCD[1]
        

        elif prtcl == 'MPT':
            hasGHZ = users_connected(G_prime,users)

            # update 19/8/24: users can act as repeaters!
            continue
            users_path_exist = []
            for nodes in combinations(users, 2):
                G_prime_no_u = G_prime.copy()
                for u in [u_ for u_ in users if u_ not in nodes]:
                    G_prime_no_u.remove_node(u)
                users_path_bool = nx.has_path(G_prime_no_u, *nodes)
                users_path_exist.append(users_path_bool)
            if all(users_path_exist):
                hasGHZ = True


    if hasGHZ:
        if paths == 'SP':
            return (G_prime,iter,entangle_edges)

        elif prtcl == 'MPS':
            return (G_prime,iter,valid_edges)

        elif prtcl == 'MPT':
            G_prime_connected = G_prime.copy()
            for node in G_prime.nodes():
                if not nx.has_path(G_prime,users[0],node): # connected to 1 user means connected to all users
                    G_prime_connected.remove_node(node)
            
            steiner = steiner_best(G_prime_connected,users)

            return (G_prime,iter,steiner)

### Read data

In [75]:
sps_iters_fp = r"data\iterations\full\SPS"
spt_iters_fp = r"data\iterations\full\SPT"
mps_iters_fp = r"data\iterations\full\MPS"
mpt_iters_fp = r"data\iterations\full\MPT"

sps_iters_dict = {}
spt_iters_dict = {}
mps_iters_dict = {}
mpt_iters_dict = {}

for file in os.listdir(sps_iters_fp):
    sps_iters_txt_name = os.fsdecode(file)
    with open(f'{sps_iters_fp}\\{file}','r') as sps_txt:
        sps_txt_data = sps_txt.read()
        sps_iters_str = sps_txt_data.split(',')
        sps_iters_l = list(sps_iters_str)
        sps_iters_l = [int(n) for n in sps_iters_l if n!='']
        index = int( sps_iters_txt_name.replace('SPS_iters_','').replace('.txt','') )
        sps_iters_dict[index] = sps_iters_l
    
for file in os.listdir(spt_iters_fp):
    spt_iters_txt_name = os.fsdecode(file)
    with open(f'{spt_iters_fp}\\{file}','r') as spt_txt:
        spt_txt_data = spt_txt.read()
        spt_iters_str = spt_txt_data.split(',')
        spt_iters_l = list(spt_iters_str)
        spt_iters_l = [int(n) for n in spt_iters_l if n!='']
        index = int( spt_iters_txt_name.replace('SPT_iters_','').replace('.txt','') )
        spt_iters_dict[index] = spt_iters_l

for file in os.listdir(mps_iters_fp):
    mps_iters_txt_name = os.fsdecode(file)
    with open(f'{mps_iters_fp}\\{file}','r') as mps_txt:
        mps_txt_data = mps_txt.read()
        mps_iters_str = mps_txt_data.split(',')
        mps_iters_l = list(mps_iters_str)
        mps_iters_l = [int(n) for n in mps_iters_l if n!='']
        index = int( mps_iters_txt_name.replace('MPS_iters_','').replace('.txt','') )
        mps_iters_dict[index] = mps_iters_l

for file in os.listdir(mpt_iters_fp):
    mpt_iters_txt_name = os.fsdecode(file)
    with open(f'{mpt_iters_fp}\\{file}','r') as mpt_txt:
        mpt_txt_data = mpt_txt.read()
        mpt_iters_str = mpt_txt_data.split(',')
        mpt_iters_l = list(mpt_iters_str)
        mpt_iters_l = [int(n) for n in mpt_iters_l if n!='']
        index = int( mpt_iters_txt_name.replace('MPT_iters_','').replace('.txt','') )
        mpt_iters_dict[index] = mpt_iters_l

sps_ET_dict = {key: float(np.mean(val[:2500])) for key, val in sps_iters_dict.items()}
spt_ET_dict = {key: float(np.mean(val[:2500])) for key, val in spt_iters_dict.items()}
mps_ET_dict = {key: float(np.mean(val[:2500])) for key, val in mps_iters_dict.items()}
mpt_ET_dict = {key: float(np.mean(val[:2500])) for key, val in mpt_iters_dict.items()}

In [76]:
mps_reps_fp = r"data\rep_usage_data\full\MPS"
mpt_reps_fp = r"data\rep_usage_data\full\MPT"

mps_reps_dict = {}
mpt_reps_dict = {}

for file in os.listdir(mps_reps_fp):
    mps_reps_txt_name = os.fsdecode(file)
    with open(f'{mps_reps_fp}\\{file}','r') as mps_reps_txt:
        mps_reps_txt_data = mps_reps_txt.read()
        mps_reps_txt_data = mps_reps_txt_data.split('\n')
        mps_reps_l = []
        for line in mps_reps_txt_data:
            line_l = line[1:-1].split(', ') # [1:-1] to remove the { and }
            line_set = {int(n) for n in line_l if n!=''}
            mps_reps_l.append(line_set)
        index = int( mps_reps_txt_name.replace('MPS_reps_','').replace('.txt','') )
        mps_reps_l = [s for s in mps_reps_l if len(s)>0]
        mps_reps_dict[index] = mps_reps_l[:2500]


for file in os.listdir(mpt_reps_fp):
    mpt_reps_txt_name = os.fsdecode(file)
    with open(f'{mpt_reps_fp}\\{file}','r') as mpt_reps_txt:
        mpt_reps_txt_data = mpt_reps_txt.read()
        mpt_reps_txt_data = mpt_reps_txt_data.split('\n')
        mpt_reps_l = []
        for line in mpt_reps_txt_data:
            line_l = line[1:-1].split(', ') # [1:-1] to remove the { and }
            line_set = {int(n) for n in line_l if n!=''}
            mpt_reps_l.append(line_set)
        index = int( mpt_reps_txt_name.replace('MPT_reps_','').replace('.txt','') )
        mpt_reps_l = [s for s in mpt_reps_l if len(s)>0]
        mpt_reps_dict[index] = mpt_reps_l[:2500]

In [81]:
sps_edges = {0: [(35, 1), (1, 36), (36, 38), (38, 14), (14, 8), (8, 5), (5, 3), (3, 2), (2, 4), (35, 23), (23, 24), (24, 25), (35, 31), (31, 17), (17, 16), (16, 15), (15, 20), (20, 11), (11, 43), (43, 10), (10, 42), (42, 41), (35, 34), (34, 22)], 1: [(2, 1), (2, 3), (2, 4), (4, 5), (2, 6)], 2: [(26, 12), (12, 10), (10, 15), (26, 17), (26, 27), (27, 20), (20, 11), (11, 24), (24, 7), (26, 28), (28, 3), (3, 4), (4, 1)], 3: [(12, 1), (1, 3), (12, 10), (10, 9), (12, 11), (12, 13), (13, 7)], 4: [(8, 1), (1, 4), (4, 16), (16, 19), (19, 18), (8, 4), (4, 15), (15, 19), (19, 17), (8, 5), (5, 6), (6, 14), (14, 13), (8, 12), (12, 11)], 5: [(1, 7), (1, 8), (8, 3), (3, 9), (1, 10), (10, 12), (12, 11)], 6: [(9, 4), (4, 2), (9, 8), (8, 3), (9, 12), (12, 11), (9, 14), (14, 10)], 7: [(12, 8), (8, 5), (5, 3), (3, 1), (1, 19), (12, 11), (11, 10), (10, 2), (12, 14), (14, 13), (13, 17), (12, 18), (18, 16)], 8: [(7, 3), (3, 2), (7, 6), (6, 4), (4, 1), (7, 8), (8, 9), (9, 10), (10, 12), (7, 10), (10, 11)], 10: [(11, 4), (4, 3), (3, 2), (11, 10), (10, 9), (9, 8), (11, 17), (17, 16), (11, 12), (12, 13), (13, 14), (14, 15)], 11: [(81, 27), (27, 21), (21, 53), (53, 132), (132, 135), (135, 134), (134, 133), (81, 58), (58, 57), (57, 60), (60, 59), (59, 54), (54, 53), (53, 131), (131, 130), (130, 33), (33, 128), (128, 113), (113, 115), (115, 116), (81, 80), (80, 64), (64, 63), (63, 62), (62, 61), (61, 71), (71, 16), (16, 19), (19, 10), (10, 8), (8, 7), (7, 6), (6, 5), (81, 82), (82, 75), (75, 97), (97, 98), (98, 68), (68, 87), (87, 88), (88, 94), (94, 125), (125, 127), (127, 47), (47, 42), (42, 41), (41, 141), (141, 140), (140, 108), (108, 44), (44, 138)], 12: [(12, 5), (12, 11), (11, 9), (9, 10), (10, 13), (12, 16), (16, 15), (12, 18), (18, 17)], 13: [(12, 11), (11, 14), (14, 10), (10, 13), (12, 16), (16, 15), (12, 18), (18, 17), (17, 6), (6, 5), (12, 22), (22, 23)], 14: [(18, 7), (7, 5), (18, 19), (18, 20), (20, 21), (21, 6), (6, 10), (10, 15), (15, 11), (11, 12), (12, 13)], 16: [(3, 1), (3, 6), (3, 7), (7, 8), (3, 4), (4, 12)], 17: [(32, 3), (3, 38), (38, 35), (35, 41), (32, 4), (4, 21), (32, 14), (14, 50), (50, 46), (46, 25), (25, 18), (32, 33), (33, 6), (6, 23), (23, 7), (7, 39), (39, 37)], 19: [(7, 1), (7, 2), (7, 5), (5, 3), (7, 6)], 21: [(12, 10), (10, 2), (2, 1), (12, 13), (12, 15), (15, 14), (12, 19), (19, 23), (23, 30)], 22: [(9, 3), (3, 1), (9, 8), (8, 7), (9, 13), (13, 12), (9, 19), (19, 18), (18, 17), (17, 20), (20, 24), (24, 25)], 23: [(25, 7), (7, 5), (5, 6), (6, 9), (9, 15), (15, 14), (14, 13), (13, 12), (12, 20), (20, 21), (25, 11), (25, 17), (17, 4), (4, 18), (18, 3), (3, 10), (10, 22), (22, 23), (25, 26)], 26: [(14, 5), (5, 4), (4, 18), (18, 17), (14, 8), (8, 11), (11, 16), (14, 9), (9, 19), (14, 15), (15, 23)], 27: [(24, 26), (26, 49), (24, 35), (35, 43), (43, 40), (24, 38), (38, 57), (57, 18), (18, 15), (15, 21), (21, 5), (5, 32), (32, 54), (54, 55), (55, 28), (24, 44), (44, 16), (16, 36), (36, 23), (23, 52), (52, 27), (27, 12), (12, 14), (14, 13), (13, 9), (9, 53), (53, 1), (1, 8)], 28: [(1, 2), (2, 14), (1, 4), (4, 3), (3, 20), (20, 19), (19, 26), (26, 24), (24, 23), (23, 22), (22, 21), (1, 7), (7, 8), (8, 11), (11, 15), (1, 10), (10, 9), (9, 6), (6, 5), (5, 16)], 29: [(22, 1), (1, 19), (19, 30), (30, 34), (34, 38), (38, 33), (33, 16), (16, 18), (18, 49), (49, 62), (62, 67), (67, 2), (2, 11), (22, 3), (3, 20), (20, 8), (8, 64), (64, 63), (22, 57), (57, 5), (5, 26), (26, 7), (7, 39), (39, 68), (68, 69), (69, 35), (22, 71), (71, 48), (48, 58), (58, 32), (32, 61)], 30: [(31, 18), (18, 15), (15, 11), (11, 10), (10, 9), (31, 19), (19, 12), (12, 13), (13, 2), (2, 1), (31, 27), (27, 39), (39, 50), (50, 49), (49, 53), (53, 90), (90, 89), (89, 97), (97, 96), (96, 95), (31, 30), (30, 28), (28, 32), (32, 36), (36, 63), (63, 64), (64, 65), (65, 70), (70, 69)], 33: [(12, 2), (2, 7), (12, 3), (3, 16), (16, 1), (12, 11), (11, 14), (14, 10), (10, 13), (12, 24), (24, 15)], 34: [(28, 27), (27, 19), (19, 18), (18, 17), (17, 16), (16, 6), (6, 4), (4, 3), (3, 1), (28, 29), (29, 34), (34, 35), (35, 36), (36, 41), (41, 44), (44, 47), (28, 30), (30, 25), (25, 24), (24, 21), (21, 16), (16, 5), (5, 2), (28, 31), (31, 37), (37, 38), (38, 39), (39, 45), (45, 46), (46, 47), (47, 48)], 35: [(11, 1), (11, 2), (11, 9), (9, 8), (8, 5), (5, 4), (4, 3), (11, 12), (12, 17), (17, 19), (19, 23), (23, 26)], 36: [(26, 7), (7, 8), (8, 1), (1, 4), (4, 3), (3, 14), (14, 21), (26, 25), (25, 24), (26, 27), (27, 28), (28, 29), (26, 30), (30, 5), (5, 6), (6, 17), (17, 10)], 37: [(13, 9), (9, 7), (7, 6), (13, 10), (10, 8), (8, 4), (4, 5), (13, 15), (13, 16), (16, 18), (18, 21)], 38: [(7, 4), (4, 3), (3, 2), (7, 5), (5, 1), (7, 8), (7, 9)], 39: [(8, 5), (5, 1), (8, 7), (7, 3), (3, 2), (2, 4), (8, 13), (8, 14)], 40: [(2, 1), (2, 4), (4, 3), (3, 12), (12, 11), (2, 7), (7, 5), (2, 8), (8, 9), (9, 18), (18, 16), (16, 17)], 41: [(14, 5), (5, 4), (4, 18), (18, 17), (14, 8), (8, 11), (11, 16), (14, 9), (9, 19), (14, 15), (15, 23)], 42: [(17, 6), (6, 5), (17, 10), (10, 11), (17, 13), (13, 14), (17, 15)], 43: [(20, 2), (2, 1), (1, 6), (6, 19), (19, 18), (20, 14), (14, 16), (16, 23), (23, 29), (20, 15), (15, 22), (22, 28), (20, 24)], 45: [(4, 1), (1, 21), (21, 2), (2, 8), (4, 16), (4, 19), (4, 27), (27, 26), (26, 9), (9, 10)], 47: [(1, 8), (8, 12), (12, 13), (13, 24), (24, 34), (34, 36), (36, 2), (1, 14), (14, 11), (1, 15), (15, 5), (5, 35), (35, 16), (1, 19), (19, 18), (18, 30)], 48: [(12, 1), (1, 7), (7, 5), (5, 6), (6, 9), (12, 2), (2, 8), (8, 10), (12, 16), (16, 15), (15, 14), (14, 13), (12, 19), (19, 11)], 49: [(32, 23), (23, 29), (29, 3), (3, 20), (20, 8), (32, 58), (58, 48), (48, 71), (71, 22), (22, 1), (1, 19), (19, 26), (26, 7), (7, 39), (39, 68), (68, 69), (69, 35), (32, 61), (61, 60), (60, 59), (59, 43), (43, 99), (99, 92), (92, 96), (32, 93), (93, 98), (98, 97)], 50: [(33, 12), (12, 11), (11, 14), (14, 20), (20, 21), (21, 22), (22, 23), (33, 24), (24, 25), (25, 3), (3, 4), (4, 1), (1, 2), (2, 7), (7, 9), (9, 13), (33, 31), (33, 32), (32, 28), (28, 27)], 51: [(14, 11), (11, 9), (9, 5), (5, 3), (14, 13), (13, 20), (20, 18), (14, 15), (15, 16), (16, 12), (12, 8), (8, 6), (6, 4)], 52: [(12, 7), (7, 47), (47, 46), (12, 10), (10, 9), (9, 36), (36, 43), (43, 42), (42, 41), (41, 40), (40, 34), (34, 15), (15, 16), (16, 17), (17, 19), (19, 20), (20, 21), (21, 29), (29, 26), (12, 37), (37, 40), (40, 23), (23, 35), (12, 49), (49, 52)], 54: [(21, 20), (20, 19), (19, 18), (18, 17), (21, 22), (22, 13), (13, 12), (12, 11), (11, 10), (10, 9), (9, 8), (8, 7), (7, 6), (6, 5), (21, 24), (24, 25), (21, 27), (27, 28), (28, 29), (29, 30), (30, 1), (1, 2), (2, 3), (3, 4)], 55: [(14, 12), (12, 22), (22, 32), (32, 33), (33, 35), (35, 29), (14, 13), (13, 1), (1, 2), (2, 7), (7, 6), (6, 5), (5, 38), (38, 30), (14, 15), (15, 16), (16, 36), (36, 45), (14, 22), (22, 25), (25, 21), (21, 24)], 57: [(31, 21), (21, 15), (15, 10), (10, 9), (9, 19), (19, 18), (18, 6), (6, 24), (24, 23), (23, 22), (22, 17), (17, 16), (16, 11), (31, 26), (26, 8), (8, 1), (1, 4), (31, 30), (30, 29), (29, 28), (28, 14), (14, 13), (31, 35)], 58: [(23, 3), (3, 26), (26, 20), (23, 4), (4, 1), (1, 2), (2, 7), (7, 8), (8, 10), (10, 14), (14, 13), (13, 19), (19, 17), (17, 16), (16, 11), (23, 22), (22, 21), (21, 5), (5, 6), (6, 9), (9, 15), (15, 12), (12, 18), (23, 24), (24, 25), (25, 27), (27, 28)], 60: [(11, 5), (11, 6), (6, 2), (2, 1), (1, 3), (3, 8), (11, 7), (11, 10), (10, 9)], 61: [(7, 6), (6, 2), (7, 8), (8, 22), (22, 25), (7, 12), (12, 5), (5, 1), (7, 17), (17, 21), (21, 26), (26, 19), (19, 23)], 62: [(4, 1), (4, 2), (2, 5), (5, 13), (13, 14), (4, 3), (4, 15), (15, 11), (11, 10)], 63: [(30, 11), (11, 23), (23, 22), (30, 21), (21, 45), (45, 44), (44, 43), (30, 31), (31, 28), (28, 29), (29, 24), (24, 15), (15, 7), (7, 6), (6, 9), (30, 33), (33, 34), (34, 1), (1, 4), (4, 3), (3, 42)], 64: [(4, 1), (4, 3), (3, 7), (7, 11), (11, 18), (4, 5), (4, 13), (13, 17)], 65: [(7, 6), (6, 5), (5, 4), (7, 8), (8, 9), (9, 3), (3, 1), (7, 10), (7, 11), (11, 13)], 68: [(9, 8), (8, 7), (9, 10), (10, 11), (11, 12), (12, 5), (5, 4), (4, 1), (9, 13), (9, 17), (17, 16), (16, 15), (15, 19)], 70: [(26, 5), (5, 1), (1, 28), (28, 37), (37, 20), (20, 11), (11, 12), (26, 6), (6, 9), (26, 24), (24, 22), (26, 27), (27, 34)], 71: [(30, 11), (11, 12), (12, 25), (25, 37), (37, 31), (30, 14), (14, 58), (58, 15), (15, 10), (10, 2), (2, 9), (30, 32), (32, 8), (8, 53), (53, 17), (17, 21), (21, 39), (39, 48), (48, 29), (29, 51), (30, 55), (55, 26), (26, 60), (60, 18), (18, 46), (46, 45), (45, 22)], 72: [(18, 10), (10, 2), (2, 4), (4, 7), (7, 8), (8, 5), (5, 6), (6, 9), (18, 15), (15, 14), (14, 13), (18, 17), (18, 19), (19, 20)], 74: [(18, 1), (1, 4), (4, 5), (5, 8), (18, 11), (11, 3), (18, 13), (13, 14), (14, 15), (18, 17)], 76: [(1, 2), (2, 8), (1, 3), (3, 4), (4, 23), (23, 15), (15, 16), (1, 4), (4, 20), (20, 18), (1, 6), (6, 9)], 77: [(12, 6), (6, 5), (5, 1), (12, 9), (9, 21), (21, 20), (20, 22), (22, 23), (23, 25), (25, 31), (31, 30), (30, 32), (12, 10), (10, 11), (11, 15), (12, 13)], 79: [(17, 14), (14, 12), (12, 1), (17, 15), (15, 18), (18, 20), (20, 21), (21, 23), (23, 25), (17, 16), (16, 11), (11, 9), (9, 7), (7, 4), (4, 2), (17, 19), (19, 22), (22, 24)], 81: [(10, 1), (1, 5), (5, 30), (30, 2), (2, 19), (19, 18), (18, 3), (10, 7), (7, 12), (12, 8), (8, 22), (22, 27), (10, 17), (17, 29), (29, 28), (28, 15), (10, 23), (23, 9), (9, 20), (20, 13)], 82: [(9, 2), (2, 1), (1, 5), (5, 4), (9, 7), (9, 10), (10, 8), (9, 17), (17, 6)], 83: [(6, 2), (2, 1), (1, 12), (6, 5), (6, 9), (6, 10), (10, 11), (11, 3), (3, 4)], 84: [(11, 5), (11, 6), (6, 14), (11, 9), (9, 4), (11, 10), (10, 7), (7, 13), (13, 1)], 86: [(6, 3), (3, 2), (6, 4), (4, 1), (1, 5), (6, 7), (7, 9), (9, 8), (6, 15), (15, 14), (14, 13), (13, 12), (12, 11), (11, 10)], 88: [(18, 9), (9, 6), (6, 5), (5, 25), (18, 13), (13, 19), (18, 22), (22, 16), (16, 15), (18, 28), (28, 29), (29, 30)], 89: [(18, 2), (2, 7), (7, 17), (18, 10), (10, 3), (3, 4), (4, 1), (18, 12), (12, 11), (18, 15), (15, 14)], 90: [(12, 13), (13, 3), (3, 2), (2, 4), (12, 14), (14, 15), (15, 16), (12, 15), (15, 9), (12, 20), (20, 19), (19, 11), (11, 10)], 92: [(15, 7), (7, 5), (5, 4), (4, 3), (3, 1), (15, 16), (16, 14), (14, 13), (13, 9), (9, 6), (6, 8), (15, 19)], 93: [(24, 26), (26, 49), (24, 35), (35, 43), (43, 40), (24, 38), (38, 57), (57, 18), (18, 15), (15, 21), (21, 5), (5, 32), (32, 54), (54, 55), (55, 28), (24, 44), (44, 16), (16, 36), (36, 23), (23, 52), (52, 27), (27, 12), (12, 14), (14, 13), (13, 9), (9, 53), (53, 1), (1, 8)], 94: [(15, 2), (15, 6), (6, 1), (15, 11), (11, 7), (7, 8), (15, 29), (29, 25), (25, 27)], 95: [(13, 6), (6, 1), (1, 2), (13, 12), (12, 11), (13, 14), (14, 15), (15, 3), (13, 18), (18, 17), (17, 16), (16, 21)], 96: [(22, 7), (7, 5), (5, 8), (22, 15), (15, 12), (22, 16), (22, 18)], 97: [(10, 9), (9, 6), (10, 13), (10, 15), (15, 17), (17, 18), (18, 19), (19, 20), (10, 16), (16, 17), (17, 3), (3, 20), (20, 11)], 98: [(16, 3), (3, 1), (1, 7), (7, 8), (16, 10), (10, 6), (6, 5), (16, 12), (16, 19)], 100: [(3, 1), (1, 6), (3, 2), (2, 11), (11, 5), (5, 9), (3, 10)], 101: [(11, 7), (7, 1), (1, 12), (11, 13), (13, 5), (5, 26), (26, 23), (11, 18), (18, 17), (17, 22), (22, 2), (11, 24), (24, 20), (20, 6), (6, 16)], 102: [(2, 1), (2, 3), (3, 4), (2, 5), (5, 6), (2, 7)], 103: [(6, 2), (6, 3), (3, 4), (4, 5), (6, 7), (6, 8)], 104: [(1, 2), (2, 8), (8, 16), (16, 7), (1, 3), (3, 10), (10, 20), (1, 5), (5, 15), (15, 19), (19, 18), (18, 17), (17, 24), (1, 25), (25, 12), (12, 23), (23, 22)]}
spt_edges = {0: [(2, 4), (2, 10), (10, 42), (10, 43), (11, 20), (11, 43), (15, 16), (15, 20), (16, 17), (17, 31), (22, 34), (23, 24), (23, 35), (24, 25), (31, 35), (34, 35), (41, 42)], 1: [(1, 2), (2, 3), (2, 6), (5, 6)], 2: [(1, 4), (3, 4), (3, 28), (5, 6), (5, 8), (6, 9), (7, 8), (9, 10), (10, 15), (13, 14), (13, 19), (14, 15), (16, 17), (16, 18), (17, 26), (18, 19), (26, 28)], 3: [(1, 3), (1, 12), (6, 7), (6, 10), (9, 10), (10, 12), (11, 12)], 4: [(4, 8), (4, 16), (6, 11), (6, 14), (8, 12), (11, 12), (13, 14), (16, 19), (17, 19), (18, 19)], 5: [(1, 7), (2, 5), (2, 7), (4, 5), (4, 9), (6, 9), (6, 13), (11, 14), (13, 14)], 6: [(1, 4), (1, 8), (2, 4), (3, 8), (4, 9), (8, 16), (9, 12), (10, 14), (11, 12), (14, 16)], 7: [(2, 10), (2, 19), (10, 11), (11, 12), (12, 18), (13, 15), (13, 17), (15, 16), (17, 18)], 8: [(1, 2), (2, 3), (3, 7), (7, 8), (8, 9), (9, 11), (11, 12)], 10: [(1, 2), (1, 5), (5, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16)], 11: [(128, 113), (128, 129), (129, 133), (3, 4), (3, 6), (4, 50), (133, 134), (134, 135), (135, 52), (5, 6), (6, 7), (7, 8), (138, 44), (140, 108), (140, 141), (141, 41), (8, 10), (10, 19), (16, 19), (16, 71), (41, 48), (44, 108), (46, 49), (46, 123), (47, 48), (47, 123), (49, 50), (52, 59), (57, 58), (57, 60), (58, 81), (59, 60), (61, 62), (61, 71), (62, 63), (63, 64), (64, 80), (80, 81), (113, 115), (115, 116)], 12: [(5, 12), (6, 11), (6, 17), (9, 10), (9, 11), (10, 13), (12, 16), (12, 18), (15, 16), (17, 18)], 13: [(5, 6), (6, 11), (10, 13), (10, 14), (11, 12), (11, 14), (12, 22), (12, 24), (15, 24), (22, 23)], 14: [(2, 4), (2, 7), (3, 4), (3, 14), (5, 7), (7, 18), (13, 14), (18, 19)], 16: [(1, 3), (3, 6), (3, 7), (7, 8), (7, 9), (9, 11), (11, 12)], 17: [(7, 23), (7, 39), (10, 17), (10, 34), (11, 36), (11, 45), (17, 20), (18, 25), (18, 31), (20, 45), (21, 44), (22, 44), (22, 23), (25, 34), (27, 31), (27, 35), (35, 41), (36, 40), (37, 39), (39, 40)], 19: [(1, 7), (2, 3), (2, 7), (6, 7)], 21: [(1, 2), (2, 9), (9, 23), (12, 13), (12, 15), (14, 15), (14, 16), (16, 17), (17, 20), (20, 22), (22, 23), (23, 30)], 22: [(1, 3), (3, 4), (3, 27), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 13), (12, 13), (25, 26), (26, 27)], 23: [(3, 10), (3, 18), (4, 17), (4, 18), (5, 6), (5, 7), (6, 9), (7, 25), (9, 15), (10, 22), (11, 25), (12, 13), (12, 20), (13, 14), (14, 15), (17, 25), (20, 21), (22, 23), (25, 26)], 26: [(1, 7), (1, 18), (7, 8), (8, 11), (11, 16), (15, 18), (15, 21), (15, 23), (17, 18), (19, 20), (20, 21)], 27: [(4, 37), (4, 59), (5, 21), (5, 32), (8, 41), (18, 45), (18, 57), (19, 42), (19, 59), (20, 33), (20, 41), (21, 45), (22, 42), (22, 60), (26, 46), (26, 49), (28, 55), (28, 60), (32, 54), (33, 34), (34, 37), (35, 43), (35, 47), (38, 46), (38, 57), (40, 43), (47, 48), (48, 49), (54, 55)], 28: [(1, 2), (1, 18), (2, 14), (5, 6), (5, 16), (6, 9), (9, 25), (11, 15), (11, 17), (13, 14), (13, 19), (17, 18), (19, 26), (21, 22), (22, 23), (23, 24), (24, 26), (25, 26)], 29: [(5, 26), (5, 57), (7, 26), (7, 39), (11, 51), (13, 27), (13, 52), (22, 57), (22, 71), (25, 31), (25, 51), (27, 46), (31, 40), (32, 58), (32, 61), (35, 69), (35, 74), (39, 68), (40, 75), (42, 52), (42, 75), (43, 55), (43, 59), (46, 74), (48, 58), (48, 71), (50, 55), (50, 63), (59, 60), (60, 61), (68, 69)], 30: [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 9), (9, 10), (10, 16), (16, 17), (17, 30), (28, 30), (28, 32), (32, 36), (36, 63), (63, 64), (64, 66), (65, 70), (65, 71), (66, 67), (67, 70), (69, 70), (71, 74), (74, 79), (79, 80), (80, 81), (81, 86), (83, 84), (83, 86), (84, 85), (85, 93), (93, 99), (95, 99)], 33: [(1, 16), (2, 7), (2, 12), (3, 12), (3, 16), (10, 13), (10, 14), (11, 12), (11, 14), (12, 24), (15, 24)], 34: [(1, 2), (2, 5), (5, 16), (16, 17), (17, 18), (18, 19), (19, 27), (27, 28), (28, 29), (29, 34), (34, 35), (35, 39), (39, 45), (44, 45), (44, 47), (47, 48)], 35: [(1, 2), (2, 4), (3, 4), (4, 5), (5, 8), (8, 9), (9, 11), (11, 12), (12, 17), (17, 19), (19, 23), (23, 26)], 36: [(1, 2), (1, 4), (2, 24), (3, 4), (3, 14), (3, 23), (5, 6), (5, 30), (6, 17), (9, 10), (9, 31), (10, 17), (14, 21), (22, 23), (22, 31), (29, 30)], 37: [(2, 3), (2, 6), (3, 4), (4, 5), (6, 7), (7, 9), (9, 13), (13, 15), (13, 16), (16, 18), (18, 21)], 38: [(1, 2), (2, 3), (3, 4), (4, 7), (7, 8), (8, 9)], 39: [(1, 2), (1, 5), (2, 4), (5, 8), (8, 13), (8, 14)], 40: [(1, 2), (1, 4), (2, 7), (3, 4), (3, 12), (5, 6), (5, 7), (6, 18), (11, 12), (16, 17), (16, 18)], 41: [(1, 7), (1, 18), (7, 8), (8, 11), (11, 16), (15, 18), (15, 21), (15, 23), (17, 18), (19, 20), (20, 21)], 42: [(1, 4), (1, 7), (3, 4), (3, 11), (5, 6), (5, 8), (6, 13), (7, 8), (13, 14), (14, 15)], 43: [(1, 2), (1, 6), (2, 20), (6, 19), (15, 20), (15, 23), (18, 19), (20, 24), (22, 23), (22, 28), (23, 29)], 45: [(4, 16), (4, 19), (4, 27), (5, 6), (5, 8), (6, 9), (9, 10), (9, 26), (26, 27)], 47: [(1, 15), (1, 19), (2, 31), (4, 9), (4, 31), (5, 10), (5, 15), (9, 17), (10, 32), (11, 19), (16, 32), (16, 35), (17, 35), (18, 19), (18, 30)], 48: [(1, 7), (1, 12), (5, 6), (5, 7), (6, 9), (9, 10), (11, 18), (12, 19), (13, 18), (18, 19)], 49: [(64, 8), (64, 63), (66, 28), (66, 33), (4, 9), (4, 27), (8, 20), (9, 38), (74, 35), (74, 46), (20, 45), (89, 91), (89, 92), (90, 95), (90, 96), (27, 46), (28, 45), (91, 94), (92, 96), (94, 97), (95, 100), (33, 38), (100, 50), (50, 63)], 50: [(9, 10), (9, 13), (10, 19), (15, 19), (15, 22), (16, 17), (16, 23), (17, 18), (18, 19), (20, 21), (20, 26), (21, 22), (26, 27), (27, 28), (28, 32), (31, 33), (32, 33)], 51: [(2, 3), (2, 4), (3, 5), (5, 9), (9, 11), (11, 14), (13, 20), (13, 14), (18, 20)], 52: [(7, 12), (7, 47), (12, 37), (15, 16), (15, 34), (16, 17), (17, 19), (19, 20), (20, 21), (21, 29), (23, 35), (23, 40), (26, 29), (34, 35), (37, 40), (46, 47), (46, 48), (48, 49), (49, 52)], 54: [(1, 2), (1, 30), (2, 3), (3, 4), (4, 5), (17, 18), (18, 19), (19, 20), (20, 21), (21, 24), (21, 27), (24, 25), (27, 28), (28, 29), (29, 30)], 55: [(2, 7), (2, 28), (5, 6), (5, 38), (6, 7), (6, 9), (9, 41), (18, 19), (18, 36), (19, 20), (20, 21), (21, 24), (28, 35), (29, 35), (30, 38), (36, 45), (41, 42), (42, 44), (44, 45)], 57: [(3, 8), (3, 27), (4, 27), (5, 6), (5, 8), (6, 18), (9, 10), (9, 19), (10, 15), (11, 12), (12, 13), (13, 14), (14, 28), (15, 21), (18, 19), (21, 31), (28, 29), (29, 30), (30, 31), (31, 35)], 58: [(3, 23), (3, 26), (5, 6), (5, 21), (6, 9), (9, 15), (11, 16), (12, 15), (12, 18), (16, 17), (17, 18), (20, 26), (20, 28), (21, 22), (22, 23)], 60: [(1, 2), (1, 3), (2, 6), (3, 8), (5, 11), (6, 11), (7, 11), (9, 10), (10, 11)], 61: [(1, 3), (2, 3), (2, 6), (6, 8), (8, 22), (19, 23), (19, 26), (21, 24), (21, 26), (22, 25), (24, 25)], 62: [(1, 4), (3, 4), (4, 15), (10, 11), (11, 13), (13, 14), (13, 15)], 63: [(1, 2), (1, 4), (1, 34), (2, 7), (3, 4), (3, 42), (6, 7), (6, 9), (11, 23), (11, 30), (21, 30), (21, 45), (22, 23), (33, 34), (33, 47), (43, 44), (43, 49), (44, 45), (47, 48), (48, 49)], 64: [(1, 4), (3, 5), (3, 7), (4, 5), (4, 13), (7, 19), (13, 17), (18, 19)], 65: [(1, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 10), (10, 11), (11, 13)], 68: [(1, 2), (2, 3), (3, 5), (5, 12), (7, 8), (8, 9), (8, 13), (9, 17), (12, 14), (14, 19), (15, 16), (15, 19), (16, 17)], 70: [(6, 9), (6, 26), (9, 30), (11, 12), (11, 20), (20, 37), (22, 24), (24, 26), (27, 32), (27, 34), (27, 35), (30, 31), (31, 32), (35, 36), (36, 37)], 71: [(2, 9), (2, 10), (5, 38), (5, 59), (7, 19), (7, 54), (9, 33), (10, 15), (12, 25), (12, 42), (13, 19), (13, 20), (15, 58), (20, 58), (22, 41), (22, 45), (25, 37), (31, 37), (33, 34), (34, 38), (35, 45), (35, 49), (41, 52), (42, 43), (43, 59), (49, 50), (50, 51), (52, 54)], 72: [(2, 4), (2, 10), (4, 7), (5, 6), (5, 8), (6, 9), (7, 8), (10, 18), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (17, 20)], 74: [(1, 4), (1, 11), (3, 11), (4, 5), (5, 8), (11, 18), (13, 14), (13, 18), (14, 15), (17, 18)], 76: [(1, 4), (1, 6), (2, 5), (2, 8), (4, 20), (5, 6), (6, 9), (15, 16), (15, 23), (18, 20), (20, 21), (21, 22), (22, 23)], 77: [(1, 2), (2, 3), (3, 4), (4, 9), (4, 21), (9, 12), (10, 11), (10, 12), (11, 15), (13, 15), (20, 21), (20, 22), (22, 23), (23, 26), (26, 28), (28, 29), (29, 30), (30, 32)], 79: [(1, 2), (2, 3), (3, 5), (5, 6), (6, 14), (14, 17), (15, 17), (15, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 24), (24, 25)], 81: [(2, 19), (2, 30), (3, 18), (5, 15), (5, 30), (9, 20), (9, 23), (10, 17), (10, 23), (13, 20), (13, 25), (15, 28), (17, 29), (18, 19), (21, 24), (21, 27), (24, 25), (28, 29)], 82: [(1, 5), (1, 6), (4, 5), (6, 17), (7, 8), (7, 9), (9, 17)], 83: [(1, 2), (1, 12), (2, 6), (3, 4), (3, 11), (5, 6), (6, 9), (11, 12)], 84: [(1, 2), (1, 14), (2, 12), (4, 9), (5, 11), (5, 12), (9, 11)], 86: [(2, 3), (2, 5), (3, 6), (6, 15), (8, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15)], 88: [(5, 6), (5, 25), (6, 9), (9, 10), (9, 18), (10, 20), (15, 16), (16, 22), (18, 22), (19, 20), (22, 26), (26, 28), (28, 29), (29, 30)], 89: [(1, 8), (7, 8), (7, 17), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17)], 90: [(2, 3), (2, 4), (3, 21), (9, 10), (9, 15), (11, 16), (11, 19), (15, 16), (19, 20), (20, 21)], 92: [(1, 3), (3, 6), (6, 8), (6, 9), (9, 13), (13, 14), (14, 16), (15, 16), (15, 19)], 93: [(4, 37), (4, 59), (5, 21), (5, 32), (8, 41), (18, 45), (18, 57), (19, 42), (19, 59), (20, 33), (20, 41), (21, 45), (22, 42), (22, 60), (26, 46), (26, 49), (28, 55), (28, 60), (32, 54), (33, 34), (34, 37), (35, 43), (35, 47), (38, 46), (38, 57), (40, 43), (47, 48), (48, 49), (54, 55)], 94: [(32, 11), (32, 25), (1, 6), (2, 15), (6, 7), (6, 15), (7, 8), (7, 11), (25, 27)], 95: [(2, 3), (3, 15), (10, 11), (10, 16), (11, 12), (12, 13), (13, 14), (14, 15), (16, 21)], 96: [(16, 22), (18, 6), (18, 22), (6, 13), (8, 13), (12, 13)], 97: [(6, 9), (9, 10), (10, 13), (10, 16), (11, 20), (16, 17), (17, 18), (18, 19), (19, 20)], 98: [(1, 4), (1, 7), (4, 10), (5, 6), (6, 10), (7, 8), (10, 16), (12, 16), (16, 19)], 100: [(1, 3), (1, 6), (3, 10), (6, 9)], 101: [(1, 12), (1, 13), (2, 4), (4, 8), (5, 9), (5, 13), (6, 16), (6, 20), (8, 26), (9, 19), (10, 12), (10, 14), (14, 20), (19, 23), (23, 26)], 102: [(1, 2), (2, 3), (3, 4), (3, 5), (5, 6), (6, 7)], 103: [(2, 6), (3, 4), (3, 6), (4, 5), (6, 7), (6, 8)], 104: [(1, 2), (1, 3), (2, 6), (3, 10), (4, 6), (4, 14), (7, 14), (7, 16), (9, 20), (9, 21), (10, 20), (13, 16), (13, 19), (17, 18), (17, 24), (18, 19), (21, 22)]}

In [88]:
sps_nodes = {}
spt_nodes = {}

for i in sps_edges:
    sps_nodes[i] = []
    for e in sps_edges[i]:
        if e[0] not in sps_nodes[i]:
            sps_nodes[i].append(e[0])
        if e[1] not in sps_nodes[i]:
            sps_nodes[i].append(e[1])

for i in spt_edges:
    spt_nodes[i] = []
    for e in spt_edges[i]:
        if e[0] not in spt_nodes[i]:
            spt_nodes[i].append(e[0])
        if e[1] not in spt_nodes[i]:
            spt_nodes[i].append(e[1])

In [77]:
mps_rep_freq = {}
mpt_rep_freq = {}

for i in mps_reps_dict:
    mps_rep_freq[i] = {}
    for node in real_topo[i].nodes():
        mps_rep_freq[i][node] = 0
        
        for set_ in mps_reps_dict[i]:
            if node in set_:
                mps_rep_freq[i][node] += 1


for i in mpt_reps_dict:
    mpt_rep_freq[i] = {}
    for node in real_topo[i].nodes():
        mpt_rep_freq[i][node] = 0
        
        for set_ in mpt_reps_dict[i]:
            if node in set_:
                mpt_rep_freq[i][node] += 1

In [78]:
print(mps_rep_freq)
print(mpt_rep_freq)

{0: {1: 2500, 2: 2425, 3: 2085, 4: 2500, 5: 2085, 6: 79, 7: 79, 8: 2085, 9: 439, 10: 2500, 11: 2480, 12: 435, 13: 435, 14: 2500, 15: 2480, 16: 2500, 17: 2500, 18: 20, 19: 533, 20: 2480, 21: 0, 22: 2500, 23: 2492, 24: 2492, 25: 2500, 26: 11, 27: 11, 28: 3, 29: 3, 30: 0, 31: 2500, 32: 843, 33: 0, 34: 2500, 35: 2500, 36: 2500, 37: 0, 38: 2500, 39: 56, 40: 0, 41: 2500, 42: 2500, 43: 2480, 44: 21, 45: 0, 46: 0}, 1: {1: 2500, 2: 2500, 3: 2500, 4: 2500, 5: 2500, 6: 2500}, 10: {1: 6, 2: 2500, 3: 2494, 4: 2500, 5: 154, 6: 564, 7: 564, 8: 2500, 9: 2352, 10: 2352, 11: 2500, 12: 2500, 13: 2500, 14: 2500, 15: 2500, 16: 2500, 17: 2500}, 100: {1: 2500, 2: 2500, 3: 2500, 4: 1397, 5: 2448, 6: 2500, 7: 289, 8: 1202, 9: 2500, 10: 2500, 11: 2005, 12: 1205}, 101: {1: 2332, 2: 2500, 3: 491, 4: 1765, 5: 2499, 6: 2009, 7: 2500, 8: 489, 9: 1313, 10: 786, 11: 2500, 12: 2500, 13: 2500, 14: 791, 15: 519, 16: 2500, 17: 942, 18: 2500, 19: 1313, 20: 2259, 21: 482, 22: 813, 23: 2500, 24: 2500, 25: 1326, 26: 1208, 27:

### Repeater removal

In [162]:
mps_iters_fp = r"data\iterations\trimmed\MPS"

mps_trimmed_real_topos = {}
# indexed by no of reps used.

for i in range(len(real_topo)):
    if i in bad_indices:
        continue

    mps_trimmed_real_topos[i] = {}

    trimmed_real_topo = real_topo[i].copy()
    trimmed_real_topo_freq_d = mps_rep_freq[i].copy()
    
    real_topo_num_reps = real_topo[i].number_of_nodes()

    print(f'\nSPS rep frac (topo {i}): ', round(len(sps_nodes[i])/real_topo_num_reps,3), f'({len(sps_nodes[i])} out of {real_topo_num_reps})')

    # initialise
    trimmed_real_topo_num_rep = real_topo[i].number_of_nodes()
    trimmed_real_topo_num_rep_frac = real_topo[i].number_of_nodes()/real_topo_num_reps
    trimmed_real_topo_num_rep_frac_s = '{:.3f}'.format(trimmed_real_topo_num_rep_frac)

    fp_mps_rep = f'data\\rep_usage_data\\trimmed\\MPS\\{i}'
    fp_mps_iter = f'data\\iterations\\trimmed\\MPS\\{i}'
    fp_mps_rep_filename = fp_mps_rep + f'\\MPS_reps_{i}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'
    fp_mps_iter_filename = fp_mps_iter + f'\\MPS_iters_{i}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'

    if not os.path.exists(fp_mps_rep):
        os.makedirs(fp_mps_rep)
    if not os.path.exists(fp_mps_iter):
        os.makedirs(fp_mps_iter)
    
    with open(fp_mps_rep_filename, 'a') as file_mps_iter, open(fp_mps_iter_filename, 'a') as file_mps_rep:
        file_mps_iter.write(str(mps_iters_dict[i])[1:-1])
        file_mps_rep.write('\n'.join(map(str, mp_reps_dict[i])))

    mps_trimmed_real_topos[i][trimmed_real_topo_num_rep] = (trimmed_real_topo.copy(),trimmed_real_topo_num_rep_frac,trimmed_real_topo_freq_d,mps_iters_dict[i])

    while min(trimmed_real_topo_freq_d.values()) != max(trimmed_real_topo_freq_d.values()):

        for node in trimmed_real_topo.copy():
            if trimmed_real_topo_freq_d[node] == min(trimmed_real_topo_freq_d.values()):
                trimmed_real_topo.remove_node(node)
        
        if not nx.is_connected(trimmed_real_topo):
            raise ValueError('Trimmed real topo is not connected')

        trimmed_real_topo_num_rep = trimmed_real_topo.number_of_nodes()
        trimmed_real_topo_num_rep_frac = trimmed_real_topo.number_of_nodes()/real_topo_num_reps
        trimmed_real_topo_num_rep_frac_s = '{:.3f}'.format(trimmed_real_topo_num_rep_frac)
        
        print(f'topo {i}, {trimmed_real_topo_num_rep_frac_s} rep frac [{trimmed_real_topo_num_rep} reps]')
    
        MPS_iter_l = []
        MPS_reps_l = []

        for _ in tqdm(range(2500)):
            
            MPS_output = protocol('MPS',trimmed_real_topo,real_topo_users[i],'flow_length')
            
            traversed_reps_mps = {n for sublist in MPS_output[2] for n in sublist}

            MPS_iter_l.append(MPS_output[1])

            MPS_reps_l.append(traversed_reps_mps)

        fp_mps_rep = f'data\\rep_usage_data\\trimmed\\MPS\\{i}'
        fp_mps_iter = f'data\\iterations\\trimmed\\MPS\\{i}'
        fp_mps_rep_filename = fp_mps_rep + f'\\MPS_reps_{i}_reps={trimmed_real_topo_num_rep}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'
        fp_mps_iter_filename = fp_mps_iter + f'\\MPS_iters_{i}_reps={trimmed_real_topo_num_rep}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'

        if not os.path.exists(fp_mps_rep):
            os.makedirs(fp_mps_rep)
        if not os.path.exists(fp_mps_iter):
            os.makedirs(fp_mps_iter)
        
        with open(fp_mps_rep_filename, 'a') as file_mps_iter, open(fp_mps_iter_filename, 'a') as file_mps_rep:
            file_mps_iter.write(str(MPS_iter_l)[1:-1])
            file_mps_rep.write('\n'.join(map(str, MPS_reps_l)))

        trimmed_real_topo_freq_d = {}
        for node in trimmed_real_topo.nodes():
            trimmed_real_topo_freq_d[node] = 0
            for set_ in MPS_reps_l:
                if node in set_:
                    trimmed_real_topo_freq_d[node] += 1

        mps_trimmed_real_topos[i][trimmed_real_topo_num_rep] = (trimmed_real_topo.copy(),trimmed_real_topo_num_rep_frac,trimmed_real_topo_freq_d, MPS_iter_l)


SPS rep frac (topo 0):  0.543 (25 out of 46)
topo 0, 0.848 rep frac [39 reps]


100%|██████████| 2500/2500 [05:01<00:00,  8.30it/s]


topo 0, 0.804 rep frac [37 reps]


100%|██████████| 2500/2500 [04:32<00:00,  9.18it/s]


topo 0, 0.783 rep frac [36 reps]


...

topo 61, 0.769 rep frac [20 reps]


100%|██████████| 2500/2500 [01:10<00:00, 35.45it/s]


topo 61, 0.692 rep frac [18 reps]


 64%|██████▍   | 1603/2500 [00:40<00:22, 39.31it/s]


KeyboardInterrupt: 

In [ ]:
mpt_iters_fp = r"data\iterations\trimmed\MPT"

mpt_trimmed_real_topos = {}
# indexed by no of reps used.

for i in range(len(real_topo)):
    if i in bad_indices or i!=12:
        continue


    mpt_trimmed_real_topos[i] = {}

    trimmed_real_topo = real_topo[i].copy()
    trimmed_real_topo_freq_d = mpt_rep_freq[i].copy()
    
    real_topo_num_reps = real_topo[i].number_of_nodes()

    print(f'\nSPT rep frac (topo {i}): ', round(len(spt_nodes[i])/real_topo_num_reps,3), f'({len(spt_nodes[i])} out of {real_topo_num_reps})')

    # initialise
    trimmed_real_topo_num_rep = real_topo[i].number_of_nodes()
    trimmed_real_topo_num_rep_frac = real_topo[i].number_of_nodes()/real_topo_num_reps
    trimmed_real_topo_num_rep_frac_s = '{:.3f}'.format(trimmed_real_topo_num_rep_frac)

    fp_mpt_rep = f'data\\rep_usage_data\\trimmed\\MPT\\{i}'
    fp_mpt_iter = f'data\\iterations\\trimmed\\MPT\\{i}'
    fp_mpt_rep_filename = fp_mpt_rep + f'\\MPT_reps_{i}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'
    fp_mpt_iter_filename = fp_mpt_iter + f'\\MPT_iters_{i}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'

    if not os.path.exists(fp_mpt_rep):
        os.makedirs(fp_mpt_rep)
    if not os.path.exists(fp_mpt_iter):
        os.makedirs(fp_mpt_iter)
    
    with open(fp_mpt_rep_filename, 'a') as file_mpt_iter, open(fp_mpt_iter_filename, 'a') as file_mpt_rep:
        file_mpt_iter.write(str(mpt_iters_dict[i])[1:-1])
        file_mpt_rep.write('\n'.join(map(str, mpt_reps_dict[i])))

    mpt_trimmed_real_topos[i][trimmed_real_topo_num_rep] = (trimmed_real_topo.copy(),trimmed_real_topo_num_rep_frac,trimmed_real_topo_freq_d,mpt_iters_dict[i])

    while min(trimmed_real_topo_freq_d.values()) != max(trimmed_real_topo_freq_d.values()):

        for node in trimmed_real_topo.copy():
            if trimmed_real_topo_freq_d[node] == min(trimmed_real_topo_freq_d.values()):
                trimmed_real_topo.remove_node(node)

        if not nx.is_connected(trimmed_real_topo):
            raise ValueError('Trimmed real topo is not connected')

        trimmed_real_topo_num_rep = trimmed_real_topo.number_of_nodes()
        trimmed_real_topo_num_rep_frac = trimmed_real_topo.number_of_nodes()/real_topo_num_reps
        trimmed_real_topo_num_rep_frac_s = '{:.3f}'.format(trimmed_real_topo_num_rep_frac)
        
        print(f'topo {i}, {trimmed_real_topo_num_rep_frac_s} rep frac [{trimmed_real_topo_num_rep} reps]')
    
        MPT_iter_l = []
        MPT_reps_l = []

        for _ in tqdm(range(2500)):
            
            MPT_output = protocol('MPT',trimmed_real_topo,real_topo_users[i],'flow_length')
            
            traversed_reps_mpt = set(MPT_output[2].nodes())

            MPT_iter_l.append(MPT_output[1])

            MPT_reps_l.append(traversed_reps_mpt)

        fp_mpt_rep = f'data\\rep_usage_data\\trimmed\\MPT\\{i}'
        fp_mpt_iter = f'data\\iterations\\trimmed\\MPT\\{i}'
        fp_mpt_rep_filename = fp_mpt_rep + f'\\MPT_reps_{i}_reps={trimmed_real_topo_num_rep}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'
        fp_mpt_iter_filename = fp_mpt_iter + f'\\MPT_iters_{i}_reps={trimmed_real_topo_num_rep}_repfrac={trimmed_real_topo_num_rep_frac_s}.txt'

        if not os.path.exists(fp_mpt_rep):
            os.makedirs(fp_mpt_rep)
        if not os.path.exists(fp_mpt_iter):
            os.makedirs(fp_mpt_iter)
        
        with open(fp_mpt_rep_filename, 'a') as file_mpt_iter, open(fp_mpt_iter_filename, 'a') as file_mpt_rep:
            file_mpt_iter.write(str(MPT_iter_l)[1:-1])
            file_mpt_rep.write('\n'.join(map(str, MPT_reps_l)))

        trimmed_real_topo_freq_d = {}
        for node in trimmed_real_topo.nodes():
            trimmed_real_topo_freq_d[node] = 0
            for set_ in MPT_reps_l:
                if node in set_:
                    trimmed_real_topo_freq_d[node] += 1

        mpt_trimmed_real_topos[i][trimmed_real_topo_num_rep] = (trimmed_real_topo.copy(),trimmed_real_topo_num_rep_frac,trimmed_real_topo_freq_d)


SPT rep frac (topo 12):  0.4583333333333333 (11 out of 24)
topo 12, 0.833 rep frac [20 reps]


100%|██████████| 20/20 [00:00<00:00, 146.73it/s]


topo 12, 0.667 rep frac [16 reps]


100%|██████████| 20/20 [00:00<00:00, 208.77it/s]


topo 12, 0.542 rep frac [13 reps]


100%|██████████| 20/20 [00:00<00:00, 245.08it/s]


topo 12, 0.458 rep frac [11 reps]


100%|██████████| 20/20 [00:00<00:00, 308.97it/s]


topo 12, 0.417 rep frac [10 reps]


100%|██████████| 20/20 [00:00<00:00, 237.80it/s]
